In [1]:
import cv2
import os
import time
import tensorflow
import numpy as np
from matplotlib import pyplot as plt

correct = cv2.VideoCapture(os.path.join('widthx08_square','correct_feed.mp4'))
incorrect = cv2.VideoCapture(os.path.join('widthx08_square','incorrect_feed.mp4'))
size = correct.get(cv2.CAP_PROP_FRAME_COUNT) + incorrect.get(cv2.CAP_PROP_FRAME_COUNT)
feed = np.empty((int(size), 180, 180, 3), dtype = 'uint8')
label = np.empty((int(size)), dtype = 'uint8')

correct_test = cv2.VideoCapture(os.path.join('widthx08_square','correct_test_1.mp4'))
incorrect_test = cv2.VideoCapture(os.path.join('widthx08_square','incorrect_test_1.mp4'))
size_test = correct_test.get(cv2.CAP_PROP_FRAME_COUNT) + incorrect_test.get(cv2.CAP_PROP_FRAME_COUNT)
test = np.empty((int(size_test), 180, 180, 3), dtype = 'uint8')
label_test = np.empty((int(size_test)), dtype = 'uint8')

In [2]:
for i in range(int(correct.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = correct.read()
    feed[i] = frame
    label[i] = 1
        
j = int(correct.get(cv2.CAP_PROP_FRAME_COUNT))
for i in range(int(incorrect.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = incorrect.read()
    feed[i + j] = frame
    label[i + j] = 0

#for i in range(int(size)):
#    cv2.imshow('feed', feed[i])
#    if cv2.waitKey(10) & 0xFF == ord('q'):
#        break
              
correct.release()
incorrect.release()
cv2.destroyAllWindows()
images = np.copy(feed)

In [3]:
for i in range(int(correct_test.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = correct_test.read()
    test[i] = frame
    label_test[i] = 1
        
j = int(correct_test.get(cv2.CAP_PROP_FRAME_COUNT))
for i in range(int(incorrect_test.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = incorrect_test.read()
    test[i + j] = frame
    label_test[i + j] = 0

#for i in range(int(size_test)):
#    cv2.imshow('test', test[i])
#    if cv2.waitKey(10) & 0xFF == ord('q'):
#        break
              
correct_test.release()
incorrect_test.release()
cv2.destroyAllWindows()
images_test = np.copy(test)

In [4]:
feed = feed.reshape((-1, 97200)).astype('float32')
feed = (feed / 255)

test = test.reshape((-1, 97200)).astype('float32')
test = (test / 255)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

model = Sequential([
   Dense(512, activation='relu', input_shape=(97200,)),
   #Dense(512, activation='relu'),
   #Dense(512, activation='relu'),
   Dense(2, activation='softmax')
])

model.compile(
   optimizer='adam',
   loss='categorical_crossentropy',
   metrics=['accuracy']
)

model.fit(
   x=feed,
   y=to_categorical(label),
   epochs=2
)

model.summary()
model.save('mlp-v2.model')

Epoch 1/2
61/61 [==============================] - 6s 93ms/step - loss: 10.8853 - accuracy: 0.8125
Epoch 2/2
61/61 [==============================] - 6s 98ms/step - loss: 3.9062e-04 - accuracy: 1.0000
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               49766912  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 49,767,938
Trainable params: 49,767,938
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: mlp.model\assets


In [6]:
eval = model.evaluate(
   test,
   to_categorical(label_test)
)
print(eval)

22/22 [==============================] - 1s 22ms/step - loss: 2.1870e-07 - accuracy: 1.0000
[2.1869705335575418e-07, 1.0]


In [7]:
def prepare(frame): #flattened
    return frame.reshape((-1, 97200))

for i in range(int(size_test)):
    if i % 100 == 0:
        prediction = model.predict(prepare(images_test[i])) 
        print(prediction)
        if prediction[0][0] < prediction[0][1]:
            cv2.imshow('correct', images_test[i])
            print('correct')
            print(label_test[i])
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            cv2.imshow('bad', images_test[i])
            print('bad')
            print(label_test[i])
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        time.sleep(5)    
        
cv2.destroyAllWindows()            
correct_test.release()
incorrect_test.release()

[[0. 1.]]
correct
1
[[0. 1.]]
correct
1
[[0. 1.]]
correct
1
[[0. 1.]]
correct
1
[[0. 1.]]
correct
1
[[1. 0.]]
bad
0
[[1. 0.]]
bad
0
